In [1]:
import os
import sys

import torch
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
from torchmetrics.classification import MulticlassF1Score
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm

/data/home/adlyamzin1/cv_contest_venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Global constants

In [2]:
HEIGHT, WIDTH = 224, 224
GLOBAL_MEAN, GLOBAL_STD = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
BATCH_SIZE = 150
# pretrained model downloaded from https://huggingface.co/google/vit-base-patch16-224
PRETRAINED_MODEL_PATH = "./vit_224"
NUM_CLASSES = 30

TRAIN_SIZE = 38_000
VAL_SIZE = 7_373

# Define Dataloaders

In [3]:
class SportsDataset(Dataset):
    def __init__(self, root_dir, labels, transform=None, is_train=True):
        super(SportsDataset, self).__init__()
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
        self.is_train = is_train
        self.id2label = False
        self.label2id = False
        
        if is_train:
            unique_labels = np.unique(self.labels['label'])
            self.id2label = {
                i: label
                for i, label in enumerate(unique_labels)
            }

            self.label2id = {
                label: i
                for i, label in enumerate(unique_labels)
            }
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        image_path = f"{self.root_dir}/{self.labels.iloc[idx, 0]}"
        
        with Image.open(image_path) as img:
            img = img.convert("RGB")
            
            if self.transform:
                img = self.transform(img)
            else:
                img.copy()
                
        if self.is_train:
            return img, self.label2id[self.labels.iloc[idx, 1]]
        
        return img

In [4]:
train_augmentations = transforms.Compose([
    transforms.Resize(size=(HEIGHT, WIDTH)),
    transforms.AutoAugment(),
    transforms.RandAugment(),
    transforms.ToTensor(),
    transforms.Normalize(mean=GLOBAL_MEAN, std=GLOBAL_STD),
])

validation_augmentation = transforms.Compose([
    transforms.Resize(size=(HEIGHT, WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize(mean=GLOBAL_MEAN, std=GLOBAL_STD),
])

In [5]:
train_pd = pd.read_csv('train.csv')
train_pd, val_pd = train_test_split(train_pd, test_size=0.2, random_state=42)
test_pd = pd.read_csv('test.csv')

train_dataset = SportsDataset('./train', train_pd, transform=train_augmentations)
val_dataset = SportsDataset('./train', val_pd, transform=validation_augmentation)
test_dataset = SportsDataset('./test', test_pd, transform=validation_augmentation, is_train=False)


train_loader = DataLoader(
    dataset=train_dataset, batch_size=BATCH_SIZE, num_workers=10, shuffle=True,
)
test_loader = DataLoader(
    dataset=test_dataset, batch_size=BATCH_SIZE, num_workers=10, shuffle=False
)
val_loader = DataLoader(
    dataset=val_dataset, batch_size=BATCH_SIZE, num_workers=10, shuffle=False
)

# Vit Transformer

In [6]:
import pytorch_lightning as pl
from transformers import ViTForImageClassification, AdamW
import torch.nn as nn

from pytorch_lightning.utilities import rank_zero_info
from pytorch_lightning.callbacks import TQDMProgressBar

In [7]:
# https://stackoverflow.com/questions/72958447/pytorch-lightning-how-can-i-output-a-summary-of-training-to-the-console
class LoggingCallback(pl.Callback):
    def on_validation_end(self, trainer: pl.Trainer, pl_module: pl.LightningModule):
        rank_zero_info("***** Test results *****")
        metrics = trainer.callback_metrics
        for key in sorted(metrics):
            if key not in ["log", "progress_bar"]:
                rank_zero_info("{} = {}\n".format(key, str(metrics[key])))

See reference tutorial on transfer learning from Hugging Face: https://github.com/NielsRogge/Transformers-Tutorials/blob/master/VisionTransformer/Fine_tuning_the_Vision_Transformer_on_CIFAR_10_with_PyTorch_Lightning.ipynb

In [8]:
class ViTSportsTransferModel(pl.LightningModule):
    def __init__(self, model_path, num_classes, id2label, label2id):
        super(ViTSportsTransferModel, self).__init__()
        self.vit = ViTForImageClassification.from_pretrained(model_path,
            num_labels=num_classes,
            id2label=id2label,
            label2id=label2id,
            ignore_mismatched_sizes=True
        )
        
        self.num_classes = num_classes
        self.metric = MulticlassF1Score(self.num_classes, average='micro')
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, pixel_values):
        outputs = self.vit(pixel_values=pixel_values)
        return outputs.logits
        
    def common_step(self, batch, batch_idx):
        pixel_values = batch[0]
        labels = batch[1]
        logits = self(pixel_values)

        
        loss = self.criterion(logits, labels)
        f1_score = self.metric(logits, labels)

        return loss, f1_score
      
    def training_step(self, batch, batch_idx):
        loss, f1_score = self.common_step(batch, batch_idx)     
        self.log("training_loss", loss)
        self.log("training_f1", f1_score)

        return loss
    
    def validation_step(self, batch, batch_idx):
        loss, f1_score = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True, prog_bar=True)
        self.log("validation_f1", f1_score, on_epoch=True, prog_bar=True)

        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=5e-5)

    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return val_loader

# Train

In [9]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping

In [10]:
model = ViTSportsTransferModel(
    num_classes=NUM_CLASSES,
    model_path=PRETRAINED_MODEL_PATH,
    id2label=train_dataset.id2label,
    label2id=train_dataset.label2id
)

trainer = Trainer(
    gpus=1,
    callbacks=[EarlyStopping(monitor='validation_loss'), LoggingCallback(),],
    max_epochs=12,
    log_every_n_steps=50,
#     enable_progress_bar=False,
)

trainer.fit(model)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at ./vit_224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([30, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([30]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/data/home/adlyamzin1/cv_contest_venv/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:479: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, usin

Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:02<00:00,  1.18s/it]

***** Test results *****
validation_f1 = tensor(0.0300, device='cuda:0')

validation_loss = tensor(3.5296, device='cuda:0')



Epoch 0:  80%|███████▉  | 242/303 [05:50<01:28,  1.45s/it, loss=0.541, v_num=45]
Validation: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 303/303 [06:22<00:00,  1.26s/it, loss=0.541, v_num=45]

***** Test results *****
training_f1 = tensor(0.8581, device='cuda:0')

training_loss = tensor(0.5790, device='cuda:0')

validation_f1 = tensor(0.9136, device='cuda:0')

validation_loss = tensor(0.3462, device='cuda:0')



...

Epoch 11:  80%|███████▉  | 242/303 [05:50<01:28,  1.45s/it, loss=0.135, v_num=45, validation_loss=0.231, validation_f1=0.940]
Validation: 0it [00:00, ?it/s]
Epoch 11: 100%|██████████| 303/303 [06:23<00:00,  1.26s/it, loss=0.135, v_num=45, validation_loss=0.231, validation_f1=0.940]

***** Test results *****
training_f1 = tensor(0.9595, device='cuda:0')

training_loss = tensor(0.1485, device='cuda:0')

validation_f1 = tensor(0.9385, device='cuda:0')

validation_loss = tensor(0.2394, device='cuda:0')



Epoch 11: 100%|██████████| 303/303 [06:23<00:00,  1.26s/it, loss=0.135, v_num=45, validation_loss=0.239, validation_f1=0.939]

`Trainer.fit` stopped: `max_epochs=12` reached.


Epoch 11: 100%|██████████| 303/303 [06:25<00:00,  1.27s/it, loss=0.135, v_num=45, validation_loss=0.239, validation_f1=0.939]


In [11]:
preds = []
model = model.to('cuda')
for batch in tqdm(test_loader):
    labels = [
        train_dataset.id2label[label_id]
        for label_id in 
        torch.argmax(model(batch.to('cuda')), dim=1).tolist()
    ]
    preds.extend(labels)
    
test_pd['label'] = preds
test_pd.to_csv('submission.csv', index=False)

100%|██████████| 130/130 [01:07<00:00,  1.94it/s]
